ZAD 4.1
Aby zmienić rozmiar filtrów w dyskryminatorze na 4x4, należy zaktualizować funkcję Discriminator() i odpowiednio zmodyfikować warstwy konwolucyjne. Zmienimy rozmiar filtrów w warstwach downsample oraz Conv2D. Zmienione rozmiary filtrów wynoszą 4x4.

In [ ]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    cloud_free_rgb = tf.keras.layers.Input(shape=[256, 256, 3], name='cloud_free_rgb')
    edge_cloud_free_rgb = tf.keras.layers.Input(shape=[256, 256, 1], name='edge_cloud_free_rgb')
    x = tf.keras.layers.concatenate([cloud_free_rgb, edge_cloud_free_rgb])

    down1 = downsample(64, 4, apply_batchnorm=False)(x)  # Zmiana rozmiaru filtra na 4x4
    down2 = downsample(128, 4)(down1)  # Zmiana rozmiaru filtra na 4x4
    down3 = downsample(128, 4)(down2)  # Zmiana rozmiaru filtra na 4x4
    down4 = downsample(256, 4)(down3)  # Zmiana rozmiaru filtra na 4x4
    down5 = downsample(256, 4)(down4)  # Zmiana rozmiaru filtra na 4x4
    down6 = downsample(512, 4)(down5)  # Zmiana rozmiaru filtra na 4x4
    down7 = downsample(512, 4)(down6)  # Zmiana rozmiaru filtra na 4x4

    zero_pad1 = tf.keras.layers.ZeroPadding2D(2)(down7) # Zmiana paddingu na 2
    conv = tf.keras.layers.Conv2D(512, 4, strides=1,  # Zmiana rozmiaru filtra na 4x4
                                  kernel_initializer=initializer,
                                  use_bias=False)(zero_pad1)

    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)
    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)
    layer10 = tf.keras.layers.Conv2D(1, 4, strides=1,  # Zmiana rozmiaru filtra na 4x4
                                     kernel_initializer=initializer)(zero_pad2)

    flatten_layer = tf.keras.layers.Flatten()(layer10)
    dense = tf.keras.layers.Dense(1)(flatten_layer)

    return tf.keras.Model(inputs=[cloud_free_rgb, edge_cloud_free_rgb], outputs=tf.keras.activations.sigmoid(dense))


ZAD 4.2
Aby uwzględniać tylko obrazy RGB bez obrazów krawędzi, należy usunąć warstwę edge_cloud_free_rgb oraz odpowiednio dostosować wejście i wyjście modelu. Teraz wejściem dla dyskryminatora jest tylko obraz RGB bez zachmurzenia cloud_free_rgb.

In [ ]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    cloud_free_rgb = tf.keras.layers.Input(shape=[256, 256, 3], name='cloud_free_rgb')
    x = cloud_free_rgb
    down1 = downsample(64, 3, False)(x)
    down2 = downsample(128, 3)(down1)
    down3 = downsample(128, 3)(down2)
    down4 = downsample(256, 3)(down3)
    down5 = downsample(256, 3)(down4)
    down6 = downsample(512, 3)(down5)
    down7 = downsample(512, 3)(down6)
    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down7)
    conv = tf.keras.layers.Conv2D(512, 3, strides=1,
                                  kernel_initializer=initializer,
                                  use_bias=False)(zero_pad1)
    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)
    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)
    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)
    layer10 = tf.keras.layers.Conv2D(1, 3, strides=1,
                                     kernel_initializer=initializer)(zero_pad2)
    flatten_layer = tf.keras.layers.Flatten()(layer10)
    dense = tf.keras.layers.Dense(1)(flatten_layer)
    return tf.keras.Model(inputs=cloud_free_rgb, outputs=tf.keras.activations.sigmoid(dense))

ZAD 4.3
W zmodyfikowanym kodzie zostało usunięte "skip-connections". Wyjście z każdej warstwy downsample nie jest już przechowywane w liście skips, a konkatenacja z odpowiednim wyjściem z warstwy upsample została usunięta z pętli. Oto zmodyfikowany Generator():

In [ ]:
def Generator():
    clouded_rgb = tf.keras.layers.Input(shape=[256, 256, 3], name='clouded_rgb')
    clouded_nir = tf.keras.layers.Input(shape=[256, 256, 1], name='clouded_nir')
    down_stack = [
        downsample(64, 4, apply_batchnorm=False),
        downsample(128, 4),
        downsample(256, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
    ]
    up_stack = [
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4),
        upsample(256, 4),
        upsample(128, 4),
        upsample(64, 4),
    ]
    initializer = tf.random_normal_initializer(0., 0.02)
    last = tf.keras.layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                           strides=2,
                                           padding='same',
                                           kernel_initializer=initializer,
                                           activation='tanh')  # (bs, 256, 256, 3)
    x = tf.keras.layers.concatenate([clouded_rgb, clouded_nir])
    # Downsampling through the model
    for down in down_stack:
        x = down(x)
    # Upsampling
    for up in up_stack:
        x = up(x)
    x = last(x)
    return tf.keras.Model(inputs=[clouded_rgb, clouded_nir], outputs=x)

Oba modele wytrenowałem na 100 epokach. Początkowo próbowałem na 500 epokach ale Google Collab przerywał mi za każdym razem około 120 epoki (przekroczenie RAMu w darmowej wersji).
Obrazy wygenerowane przez pierwszy modelu ze "skips connections" są w katalogu: results_1.
Obrazy wygenerowane przez drugi model z usuniętym "skips connections" są w katalogu: results_2.
Skrpt wyświetlający to przejrzyście przedstawiam ponizej:

In [ ]:
import os
import matplotlib.pyplot as plt

model1_dir = 'results_1'
model2_dir = 'results_2'

num_images = 100

plt.figure(figsize=(15, 200))

for i in range(1, num_images + 1):
    image_path_model1 = os.path.join(model1_dir, f'generated_cloud_free_rgb100_{i:03d}.png')
    image_path_model2 = os.path.join(model2_dir, f'generated_cloud_free_rgb100_{i:03d}.png')

    image_model1 = plt.imread(image_path_model1)
    image_model2 = plt.imread(image_path_model2)

    plt.subplot(num_images, 2, 2*i-1)
    plt.imshow(image_model1)
    plt.title(f'Model {1}, zdjęcie {i}')
    plt.axis('off')

    plt.subplot(num_images, 2, 2*i)
    plt.imshow(image_model2)
    plt.title(f'Model {2}, zdjęcie {i}')
    plt.axis('off')

plt.tight_layout()
plt.show()


Jak widać na powyższych obrazach, obrazy wygenerowane przez model ze "skip-connections" są zdecydowanie bardziej realistyczne. Widać na nich rozróżnienie typów otoczenia (miasto, pole, las droga itp). Natomiast obrazy wygenerowane przez model bez "skip-connections" są "wyciągnięte z kapelusza". Od razu widać że model bez "skip-connections" generuje do każdego zdjęcia praktycznie idenytyczny obraz (coś w stylu gęsto zabudowanego miasta). Prowadzi to do wniosku, że model bez "skip-connections" nie nauczył się poprawnie rozpoznawać obrazów i praktycznie nie rozróżnia zdjęć między sobą. Na szczęscie model pierwszy ze "skip-connections" rozróżnia zdjęcia, uczy się i generuje coraz to dokładniejesze obrazy.
Usunięcie skip connections z generatora prowadzi do poważnych negatywnych konsekwencji i uniemożliwia sukcesywne trenowanie modelu.